In [3]:
import pathlib
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
from torchsummary import summary
from matplotlib import rc

%matplotlib inline

%load_ext autoreload
%autoreload 2

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

In [4]:
n = 1
ids_all = []

for _ in range(n):
    m = torch.randint(1, 2, (1,))
    vals = 2 * torch.rand(m).round().int() - 1
    ids_all.append(vals)

subset_size = 10

pos_mask = torch.cat([ids > 0 for ids in ids_all])
subset_mask = pos_mask
n_pos = torch.sum(pos_mask)
n_rem_neg = max(subset_size - n_pos, 0)
if n_rem_neg > 0:
    neg_mask = ~pos_mask
    neg_idxs = torch.atleast_1d(torch.squeeze(neg_mask.nonzero()))
    rand_idxs = torch.randperm(len(neg_idxs))[:n_rem_neg]
    neg_mask_subset = torch.zeros_like(neg_mask)
    neg_idxs_subset = neg_idxs[rand_idxs]
    neg_mask_subset[neg_idxs_subset] = True
    subset_mask = subset_mask | neg_mask_subset
    print(torch.sum(neg_mask_subset))

subset_mask, torch.sum(pos_mask), torch.sum(subset_mask)


tensor(1)


(tensor([True]), tensor(0), tensor(1))

In [5]:
t = torch.as_tensor([1, 0, 1, 1, 0, 1]).int()
t

tensor([1, 0, 1, 1, 0, 1], dtype=torch.int32)

In [6]:
n_rem_neg = 2
neg_idxs = torch.squeeze(t.nonzero())
rand_idxs = torch.randperm(len(neg_idxs))[:n_rem_neg]
neg_idxs_subset = neg_idxs[rand_idxs]
neg_mask = t.clone()
neg_mask[neg_idxs_subset] = False
t, neg_idxs_subset, neg_mask

(tensor([1, 0, 1, 1, 0, 1], dtype=torch.int32),
 tensor([2, 5]),
 tensor([1, 0, 0, 1, 0, 0], dtype=torch.int32))

In [8]:
m = torch.as_tensor([1, 0, 1, 0]).float()
b = torch.as_tensor(m, dtype=torch.bool)
torch.squeeze(b.nonzero())

tensor([0, 2])

In [88]:
B = 4
C = 2
S = 2

data = torch.rand((B, C, S, S))
src = torch.ones((B // 2, C, S, S))
index = torch.as_tensor([0, 2])
res_1 = torch.index_add(input=data, dim=0, index=index, source=src)
res_2 = data.clone()
res_2[index, ...] += src
torch.allclose(res_1, res_2)

True

In [95]:
a = torch.as_tensor([1, 0, 0, 1, 0, 1]).float()
b = torch.as_tensor([1, 0, 0, 1, 0]).float()
c = torch.as_tensor([1, 0, 0, 1]).float()
t = torch.cat([e for e in [a, b, c]]).bool()
t

tensor([ True, False, False,  True, False,  True,  True, False, False,  True,
        False,  True, False, False,  True])